In [1]:
using Pkg; Pkg.update()
#using Pkg; Pkg.resolve()
using SplitApplyCombine
using ParallelTemperingMonteCarlo
using DelimitedFiles
using StaticArrays
using Random, Plots

using BenchmarkTools

    Updating registry at `~/.julia/registries/General`


    Updating git-repo `https://github.com/JuliaRegistries/General.git`


   Installed ConstructionBase ──── v1.5.4
   Installed MakieCore ─────────── v0.6.6
   Installed ADTypes ───────────── v0.2.2
   Installed RecursiveArrayTools ─ v2.38.8


   Installed StaticArrays ──────── v1.6.3
   Installed SciMLBase ─────────── v1.97.0


    Updating `~/ParallelTemperingMonteCarlo.jl/Project.toml`
  [90137ffa] ↑ StaticArrays v1.6.2 ⇒ v1.6.3
    Updating `~/ParallelTemperingMonteCarlo.jl/Manifest.toml`


...

  ✓ InteratomicPotentials
  ✓ ReverseDiff


  ✓ ComponentArrays


  ✓ DftFunctionals


  ✓ DFTK


  ✓ ParallelTemperingMonteCarlo
  30 dependencies successfully precompiled in 178 seconds. 108 already precompiled.


# Main Goal

Broadly speaking, each potential requires different variables, vectors and changes at each step, this leads to method double-ups (with nonspecific variable types) as well as the need to rewrite high level functions such as mc_step and mc_cycle. By writing a single vector or struct that contains all the required vectors etc would solve both problems. 

In [2]:
abstract type PotentialVariables end

Each potential requires different vectors, both for sampling and as temporary delta vectors. Starting with the ELJ potential, requiring essentially nothing, maybe the new_dist2_vector if anything


In [3]:
mutable struct DimerPotentialVariables <: PotentialVariables
    new_dist2_vec::Vector
end
function set_variables(config,dist_2_mat,pot::AbstractDimerPotential)
    
    new_dist2_vector = zeros(length(config))
    return DimerPotentialVariable(new_dist2_vector)
end

set_variables (generic function with 1 method)

The NNPState struct becomes redundant, since we can add the various vectors to the potential variables

In [4]:
mutable struct NNPVariables <: PotentialVariables
    new_dist2_vec::Vector
    new_en_atom::Vector
    g_matrix::Array
    f_matrix::Array
    new_g_matrix::Array
    new_f_vec::Vector
end
function set_variables(config,dist2_mat,pot::RuNNerPotential)
    
    n_atoms = length(config)
    f_matrix = cutoff_function.(sqrt.(dist2_mat),Ref(pot.r_cut))
    g_matrix = total_symm_calc(config.pos,dist2_mat,f_matrix,pot.symmetryfunctions)
    
    return NNPVariables(zeros(n_atoms),zeros(n_atoms),g_matrix,f_matrix,zeros(length(pot.symmetryfunctions)), zeros(n_atoms))
end


set_variables (generic function with 2 methods)

Similarly, the embedded atom model is defined as

In [5]:
mutable struct EmbeddedAtomVariables <: PotentialVariables
    new_dist2_vec::Vector
    component_vector::Matrix
    new_component_vector::Matrix
end
function set_variables(config,dist2_matrix,pot::EmbeddedAtomPotential)

    return EmbeddedAtomVariables(zeros(length(config)),component_vector,new_component_vector)
end

set_variables (generic function with 3 methods)

And the Magnetic Field potential

In [6]:
mutable struct ELJPotentialB <: PotentialVariables
    new_dist2_vec::Vector
    tanmat::Array
    new_tan_vec::Vector
end
function set_variables(config::Config,dist2_matrix::Matrix,pot::AbstractDimerPotentialB)
    n_atoms = length(config)
    tan_matrix = get_tantheta_mat(config,config.pos)

    return ELJPotentialB(zeros(n_atoms),tan_matrix,zeros(n_atoms))
end

set_variables (generic function with 4 methods)

Next process is to consider exactly which functions depend on exactly what parameters. Discussion with Elke required on whether to split the definitions of structs into its own sub-module.